# Estudo de Self-Attention

### Gerando Inputs

In [1]:
import torch
import numpy as np

Sentenças Fictícias:

- o mikas ta ali
- eu gosto de batata frita
- existem elefantes

Informações
- O tamanho da maior sentença é 5.
- Temos 3 exemplos de teste.
- Usaremos inteiros para representar cada palavra.
- Usaremos 0 para representar os espaços vazios das sentenças.
- O tamanho do nosso vocabulário é 12 pois temos 12 palavras distintas.


In [2]:
sentence_matrix = torch.LongTensor([
    [1,   2,  3,  4, 0], #       o     mikas   ta      ali
    [5,   6,  7,  8, 9], #      eu     gosto   de   batata   frita
    [10, 11,  0,  0, 0], # existem elefantes

])
qnt_sentences, max_sentence_len = sentence_matrix.shape
print(f"Temos {qnt_sentences} sentenças e a maior sentença tem {max_sentence_len} tokens.")
sentence_matrix.shape

Temos 3 sentenças e a maior sentença tem 5 tokens.


torch.Size([3, 5])

Em geral não usamos representações inteiras para representar palavras, mas sim embedding.

Geraremos embeddings fictícios de tamanho 10 para os 12 tokens disponíveis no vocabulário.

In [3]:
vocab_size = len(sentence_matrix.unique())
embedding_size = 10
embeddings = np.random.random((vocab_size, embedding_size))
# Convertendo para tensor:
embeddings = torch.Tensor(embeddings)
print(f"Temos {max_sentence_len} embeddings de tamanho {embedding_size}.")

Temos 5 embeddings de tamanho 10.


Agora geraremos a matriz de input para os modelos.

- Nela teremos o shape 3, 5, 10 que representam as 3 sentenças com no máximo 5 tokens.
- Cada token é representado por um embedding de tamanho 10.

In [4]:
indexes = sentence_matrix
input_matrix = embeddings[indexes]
print("Temos {0} instancias de no máximo {1} tokens representados por embeddings de tamanho {2}.".format(*input_matrix.shape))
input_matrix.shape

Temos 3 instancias de no máximo 5 tokens representados por embeddings de tamanho 10.


torch.Size([3, 5, 10])

### Passando inputs pela camada de RNN:

A atenção é gerada a partir dos outputs gerados pela camada da RNN, portanto passaremos o input pela camada de RNN:

A nova saída da RNN vai ter shape (3, 5, 100) onde 3 é o número de batches, 5 é o tamanho máximo da sequência e 4 é o tamanho da representação gerada pela RNN (hidden_size).

In [5]:
rnn = torch.nn.RNN(input_size=embedding_size, hidden_size=4, batch_first=True)
output, _ = rnn(input_matrix)
print("Temos {0} inputs de tamanho máximo {1} representado por {2} dimensões.".format(*output.shape))

Temos 3 inputs de tamanho máximo 5 representado por 4 dimensões.


In [6]:
linear = torch.nn.Linear(4, 4, bias=False)
output, linear.weight.T, linear(output)

(tensor([[[ 0.1327, -0.2652,  0.1166,  0.5217],
          [ 0.4505, -0.4592, -0.0879,  0.5919],
          [-0.1805, -0.5543, -0.1924,  0.4813],
          [-0.3260, -0.5714, -0.0638,  0.3516],
          [-0.1919, -0.2689,  0.0424,  0.6110]],
 
         [[-0.3055,  0.0515,  0.1328,  0.8655],
          [-0.3236, -0.5573,  0.3173,  0.7333],
          [-0.2653, -0.5605, -0.2684,  0.6069],
          [-0.6161, -0.3906, -0.2380,  0.2492],
          [-0.5688,  0.1714, -0.3716,  0.6469]],
 
         [[ 0.2037, -0.4912, -0.1436,  0.5007],
          [ 0.2306, -0.4825,  0.1618,  0.6633],
          [-0.0481, -0.6419,  0.3294,  0.6914],
          [-0.1996, -0.6144,  0.1493,  0.7116],
          [-0.2556, -0.5254,  0.1035,  0.6727]]], grad_fn=<TransposeBackward1>),
 tensor([[ 0.2605,  0.2417, -0.4428,  0.0173],
         [-0.3816,  0.3981, -0.4339,  0.2331],
         [ 0.3449, -0.3089,  0.0009,  0.0886],
         [ 0.3689,  0.4550, -0.4122, -0.4445]], grad_fn=<PermuteBackward>),
 tensor([[[ 0.3684,  0.1